In [ ]:
# %load utils.py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import interp

import torch
import torch.nn as nn
import dgl

# 数据读取
def load_data(directory, random_seed):

    D_SSM=np.load(directory + '/DiseaseSim.npy')
    M_FSM = np.loadtxt(directory + '/M_FSM.txt')
    M_GSM = np.loadtxt(directory + '/M_GSM.txt')
    # IL = np.loadtxt(directory + '/lncRNA-fuction.txt')
    IL = np.load(directory + '/LncRNASim.npy')

    all_associations = pd.read_csv(directory + '/miRNA-disease.csv', names=['miRNA', 'disease', 'label'])  #标签： md是否有关联 0/1
    ml_associations = pd.read_csv(directory + '/miRNA-lncRNA.csv', names=['miRNA', 'lncRNA', 'label'])
    ld_associations = pd.read_csv(directory + '/lncRNA-disease.csv', names=['disease', 'lncRNA', 'label'])
    # ld_associations = pd.read_csv(directory + '/case251.csv', names=['disease', 'lncRNA', 'label'])
    ll_associations = pd.read_csv(directory + '/lnc_association.csv', names=['lncRNA', 'lncRNAT', 'label'])
    dd_associations = pd.read_csv(directory + '/dis_association.csv', names=['disease', 'diseaseT', 'label'])
    # D_SSM = (D_SSM1 + D_SSM2) / 2
    ID = D_SSM
    IM = M_FSM

    for i in range(M_FSM.shape[0]):              #miRNA同理
        for j in range(M_FSM.shape[1]):
            if IM[i][j] == 0:
                IM[i][j] = M_GSM[i][j]
# 筛选miRNA-disease正样本和与正样本数相同的负样本
#     known_associations1 = all_associations.loc[all_associations['label'] == 1]   #找到标签为1的所有值
#     unknown_associations1 = all_associations.loc[all_associations['label'] == 0]
#     random_negative1 = unknown_associations1.sample(n=known_associations1.shape[0], random_state=random_seed, axis=0)  #生成与正样本个数相同的负样本
    known_associations = ld_associations.loc[ld_associations['label'] == 1]   #找到标签为1的所有值
    unknown_associations = ld_associations.loc[ld_associations['label'] == 0]
    random_negative = unknown_associations.sample(n=known_associations.shape[0], random_state=random_seed, axis=0)  #生成与正样本个数相同的负样本

# 筛选miRNA-lncRNA和disease-lncRNA已知关联


    known_associations1=ld_associations.loc[ld_associations['label'] == 1]
    lm_associations1 = ml_associations.loc[ml_associations['label'] == 1]
    dm_associations1 = all_associations.loc[all_associations['label'] == 1]
    ll_associations1 = ll_associations.loc[ll_associations['label'] == 1]
    dd_associations1 = dd_associations.loc[dd_associations['label'] == 1]
    sample_df = known_associations.append(random_negative)       #正负样本连接  ld
# 指针重置

    sample_df.reset_index(drop=True, inplace=True)
    known_associations1.reset_index(drop=True, inplace=True)
    lm_associations1.reset_index(drop=True, inplace=True)
    dm_associations1.reset_index(drop=True, inplace=True)
    ll_associations1.reset_index(drop=True, inplace=True)
    dd_associations1.reset_index(drop=True, inplace=True)
    samples = sample_df.values                               # 获得重新编号的新样本
    known_associations=known_associations1.values
    lm_associations = lm_associations1.values
    dm_associations = dm_associations1.values
    ll_associations = ll_associations1.values
    dd_associations = dd_associations1.values


    # return ID, IM, IL, samples, ml_associations, ld_associations      # 未知关联数量较多，选择和已知关联数目相同的未知关联组成样本
    return ID, IM, IL, samples, lm_associations, dm_associations ,ll_associations ,dd_associations,known_associations

# miRNA-disease异质图和miRNA-disease-lncRNA异质图的构建
def build_graph(directory, random_seed):
    ID, IM, IL, samples, lm_associations, dm_associations ,ll_associations ,dd_associations,ld_association= load_data(directory, random_seed)  #特征与md ml ld关联
    # miRNA-disease二元异质图


    g = dgl.DGLGraph()
    g.add_nodes(ID.shape[0] + IL.shape[0])   #d和m的特征总数量 即节点数
    node_type = torch.zeros(g.number_of_nodes(), dtype=torch.int64)
    node_type[: ID.shape[0]] = 1
    g.ndata['type'] = node_type
# 0-382设为疾病节点，并传入特征
    d_sim = torch.zeros(g.number_of_nodes(), ID.shape[1])
    d_sim[: ID.shape[0], :] = torch.from_numpy(ID.astype('float32'))
    g.ndata['d_sim'] = d_sim
# 383-877设为miRNA节点，并传入特征
    l_sim = torch.zeros(g.number_of_nodes(), IL.shape[1])
    l_sim[ID.shape[0]: ID.shape[0]+IL.shape[0], :] = torch.from_numpy(IL.astype('float32'))
    g.ndata['l_sim'] = l_sim

#两个同构图
    g_l = dgl.DGLGraph()
    g_d = dgl.DGLGraph()
    g_l.add_nodes(IL.shape[0])  # m的数量 即节点数
    node_type1 = torch.zeros(g_l.number_of_nodes(), dtype=torch.int64)
    # node_type1[: IL.shape[0]] = 1
    g_l.ndata['type'] = node_type1
    ll_sim = torch.zeros(g_l.number_of_nodes(), IL.shape[1])
    ll_sim[:IL.shape[0], :] = torch.from_numpy(IL.astype('float32'))
    g_l.ndata['l_sim'] = ll_sim

    g_d.add_nodes(ID.shape[0])  # d的数量 即节点数
    node_type2 = torch.zeros(g_d.number_of_nodes(), dtype=torch.int64)
    node_type2[: ID.shape[0]] = 1
    g_d.ndata['type'] = node_type2
    dd_sim = torch.zeros(g_d.number_of_nodes(), ID.shape[1])
    dd_sim[:ID.shape[0], :] = torch.from_numpy(ID.astype('float32'))
    g_d.ndata['d_sim'] = dd_sim
# 让指针从0开始，原本节点标签从1开始
    disease_ids = list(range(1, ID.shape[0]+1))
    lncrna_ids = list(range(1, IL.shape[0]+1))

    disease_ids_invmap = {id_: i for i, id_ in enumerate(disease_ids)}   #字典  键为序号
    lncrna_ids_invmap = {id_: i for i, id_ in enumerate(lncrna_ids)}


#异构图的节点  样本节点
    sample_disease_vertices = [disease_ids_invmap[id_] for id_ in samples[:, 0]]  #samples正负样本链接      得到样本的疾病结点的值
    sample_lncrna_vertices = [lncrna_ids_invmap[id_] + ID.shape[0] for id_ in samples[:, 1]]
#添加边
    g.add_edges(sample_disease_vertices, sample_lncrna_vertices,
                data={'label': torch.from_numpy(samples[:, 2].astype('float32'))})
    g.add_edges(sample_lncrna_vertices, sample_disease_vertices,
                data={'label': torch.from_numpy(samples[:, 2].astype('float32'))})
    g.readonly=True
#同构图添加边，添加样本节点和前knn关联的节点
    lnc_V = sample_lncrna_vertices.copy()
    for i, v in enumerate(sample_lncrna_vertices):
        lnc_V[i] = v - ID.shape[0]

    g_l.add_edges(lnc_V, lnc_V,
                data={'label': torch.from_numpy(ll_associations[:, 2].astype('float32'))})
    g_d.add_edges(sample_disease_vertices, sample_disease_vertices,
                data={'label': torch.from_numpy(dd_associations[:, 2].astype('float32'))})
    g_l.readonly = True
    g_d.readonly = True





    # miRNA-disease-lncRNA三元异质图
    g0 = dgl.DGLGraph()
    g0.add_nodes(ID.shape[0] + IL.shape[0] + IM.shape[0])
    node_type = torch.zeros(g0.number_of_nodes(), dtype=torch.int64) # 返回一个878全为0的tensor 878+467=1345
    node_type[: ID.shape[0]] = 1            # disease383标记为1，miRNA标记为0，lncRNA标记为2
    node_type[ID.shape[0] + IL.shape[0]:] = 2# disease383标记为1，miRNA标记为2，lncRNA标记为0
    # node_type[ID.shape[0] : IL.shape[0]] = 0# disease383标记为1，miRNA标记为2，lncRNA标记为0
    g0.ndata['type'] = node_type             # 将图中疾病的节点记为1   给所有节点赋值


    d_sim = torch.zeros(g0.number_of_nodes(), ID.shape[1])       # （1345，383）
    d_sim[: ID.shape[0], :] = torch.from_numpy(ID.astype('float32'))    #赋疾病特征
    g0.ndata['d_sim'] = d_sim

    l_sim = torch.zeros(g0.number_of_nodes(), IL.shape[1])       # （1345,495）
    l_sim[ID.shape[0]: ID.shape[0]+IL.shape[0], :] = torch.from_numpy(IL.astype('float32'))
    g0.ndata['l_sim'] = l_sim        # 每一行表示一个miRNA的特征

    m_sim = torch.zeros(g0.number_of_nodes(), IM.shape[1])       # （1345,64）
    m_sim[ID.shape[0]+IL.shape[0]: ID.shape[0]+IL.shape[0]+IM.shape[0], :] = torch.from_numpy(IM.astype('float32'))
    g0.ndata['m_sim'] = m_sim
    # # lncRNA标签
    # lncrna_ids = list(range(1, IL.shape[0]+1))
    # lncrna_ids_invmap = {id_: i for i, id_ in enumerate(lncrna_ids)}
    # miRNA标签
    mirna_ids = list(range(1, IM.shape[0]+1))
    mirna_ids_invmap = {id_: i for i, id_ in enumerate(mirna_ids)}
    #
    dl_lncrna_vertices = [lncrna_ids_invmap[id_] + ID.shape[0] for id_ in ld_association[:, 1]]
    dl_disease_vertices = [disease_ids_invmap[id_] for id_ in ld_association[:, 0]]

    lm_lncrna_vertices = [lncrna_ids_invmap[id_] + ID.shape[0] for id_ in lm_associations[:, 1]]
    lm_mirna_vertices = [mirna_ids_invmap[id_] + ID.shape[0] + IL.shape[0] for id_ in lm_associations[:, 0]]
    # dm_mirna_vertices = [disease_ids_invmap[id_] for id_ in dm_associations[:, 1]]
    # dm_disease_vertices = [mirna_ids_invmap[id_] + ID.shape[0] + IL.shape[0] for id_ in dm_associations[:, 0]]
    dm_disease_vertices = [disease_ids_invmap[id_] for id_ in dm_associations[:, 1]]
    dm_mirna_vertices = [mirna_ids_invmap[id_] + ID.shape[0] + IL.shape[0] for id_ in dm_associations[:, 0]]

    #重新构造g0，包含元路径，用metapath函数提取同构图进行计算，取值为1的构造边

    g0 = dgl.heterograph({
        ('l', 'lm', 'm'): (torch.tensor(lm_lncrna_vertices), torch.tensor(lm_mirna_vertices)),
        ('m', 'ml', 'l'): (torch.tensor(lm_mirna_vertices), torch.tensor(lm_lncrna_vertices)),
        ('d', 'dm', 'm'): (torch.tensor(dm_disease_vertices), torch.tensor(dm_mirna_vertices)),
        ('m', 'md', 'd'): (torch.tensor(dm_mirna_vertices), torch.tensor(dm_disease_vertices)),
        ('d', 'dl', 'l'): (torch.tensor(sample_disease_vertices), torch.tensor(sample_lncrna_vertices)),
        ('l', 'ld', 'd'): (torch.tensor(sample_lncrna_vertices), torch.tensor(sample_disease_vertices))

    })
    print(g0.num_nodes('l'))
    empty1=np.zeros((412,240))
    empty2=np.zeros((652,495))
    lsimn=np.vstack((empty1,IL))
    mism=np.vstack((empty2,IM))
    g0.nodes['l'].data['l_sim'] = torch.from_numpy(lsimn.astype('float32'))
    g0.nodes['m'].data['m_sim'] = torch.from_numpy(mism.astype('float32'))
    g0.nodes['d'].data['d_sim'] = torch.from_numpy(ID.astype('float32'))
    # print(g0.nodes['d'].data['d_sim'])

    new_g = dgl.metapath_reachable_graph(g0, ['lm', 'ml'])
    print(new_g)


    g0.readonly=True

    return g, g0, g_l, g_d ,sample_disease_vertices, sample_lncrna_vertices, ID, IM, IL, samples, lm_associations, dm_associations ,ll_associations ,dd_associations


def weight_reset(m):
    if isinstance(m, nn.Linear):
        m.reset_parameters()


def plot_auc_curves(fprs, tprs, auc, directory, name):
    mean_fpr = np.linspace(0, 1, 20000)
    tpr = []

    for i in range(len(fprs)):
        tpr.append(interp(mean_fpr, fprs[i], tprs[i]))
        tpr[-1][0] = 0.0
        plt.plot(fprs[i], tprs[i], alpha=0.4, linestyle='--', label='Fold %d AUC: %.4f' % (i + 1, auc[i]))

    mean_tpr = np.mean(tpr, axis=0)
    mean_tpr[-1] = 1.0
    # mean_auc = metrics.auc(mean_fpr, mean_tpr)
    mean_auc = np.mean(auc)
    auc_std = np.std(auc)
    plt.plot(mean_fpr, mean_tpr, color='BlueViolet', alpha=0.9, label='Mean AUC: %.4f $\pm$ %.4f' % (mean_auc, auc_std))

    plt.plot([0, 1], [0, 1], linestyle='--', color='black', alpha=0.4)

    # std_tpr = np.std(tpr, axis=0)
    # tpr_upper = np.minimum(mean_tpr + std_tpr, 1)
    # tpr_lower = np.maximum(mean_tpr - std_tpr, 0)
    # plt.fill_between(mean_fpr, tpr_lower, tpr_upper, color='LightSkyBlue', alpha=0.3, label='$\pm$ 1 std.dev.')

    plt.xlim([-0.05, 1.05])
    plt.ylim([-0.05, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC curve')
    plt.legend(loc='lower right')
    plt.savefig(directory+'/%s.jpg' % name, dpi=1200, bbox_inches='tight')
    plt.close()


def plot_prc_curves(precisions, recalls, prc, directory, name):
    mean_recall = np.linspace(0, 1, 20000)
    precision = []

    for i in range(len(recalls)):
        precision.append(interp(1-mean_recall, 1-recalls[i], precisions[i]))
        precision[-1][0] = 1.0
        plt.plot(recalls[i], precisions[i], alpha=0.4, linestyle='--', label='Fold %d AP: %.4f' % (i + 1, prc[i]))

    mean_precision = np.mean(precision, axis=0)
    mean_precision[-1] = 0
    # mean_prc = metrics.auc(mean_recall, mean_precision)
    mean_prc = np.mean(prc)
    prc_std = np.std(prc)
    plt.plot(mean_recall, mean_precision, color='BlueViolet', alpha=0.9,
             label='Mean AP: %.4f $\pm$ %.4f' % (mean_prc, prc_std))  # AP: Average Precision

    plt.plot([1, 0], [0, 1], linestyle='--', color='black', alpha=0.4)

    plt.xlim([-0.05, 1.05])
    plt.ylim([-0.05, 1.05])
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('PR curve')
    plt.legend(loc='lower left')
    plt.savefig(directory + '/%s.jpg' % name, dpi=1200, bbox_inches='tight')
    plt.close()

In [ ]:
# %load model1.py
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
import numpy as np
# from mxnet import ndarray as nd
# from mxnet.gluon import nn as ng
from Hlayers import MultiHeadGATLayerL, MultiHeadGATLayerD
from layers import MultiHeadGATLayer, HAN_metapath_specific


# 语义层注意力
# 主要用于修改数据，作比较模型，这里主要有两个地方：1.hidden_size; 2.beta
# 1是语义层向量q的维度变化对参数的影响；2是beta用于语义层贡献相同时的结果
class SemanticAttention(nn.Module):
    def __init__(self, in_size, hidden_size=128):
        super(SemanticAttention, self).__init__()

        self.project = nn.Sequential(
            nn.Linear(in_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, 1, bias=False)
        )

    def forward(self, z):                               # [x, 2, 512]
        w = self.project(z).mean(0)  # 权重矩阵，获得元路径的重要性 [2, 1]
        # beta = torch.softmax(w, dim=0)
        beta = torch.sigmoid(w)
        beta = beta.expand((z.shape[0],) + beta.shape)  # [x,2,1]
        # delta = beta * z
        return (0.5 * z).sum(1)                        # [x, 512]

class HGANMDA(nn.Module):  # 1
    def __init__(self, G, GL,GD,meta_paths_list, feature_attn_size, num_heads, num_diseases, num_mirnas, num_lncrnas,
                 d_sim_dim, m_sim_dim, l_sim_dim, out_dim, dropout, slope):
        super(HGANMDA, self).__init__()

        self.G = G
        self.meta_paths = meta_paths_list
        self.num_heads = num_heads
        self.num_diseases = num_diseases
        self.num_mirnas = num_mirnas
        self.num_lncrnas = num_lncrnas

        self.gat = MultiHeadGATLayer(G, feature_attn_size, num_heads, dropout, slope)
        self.gatL = MultiHeadGATLayerL(G, feature_attn_size, num_heads, dropout, slope)
        self.gatD = MultiHeadGATLayerD(G, feature_attn_size, num_heads, dropout, slope)
        self.heads = nn.ModuleList()

        self.metapath_layers = nn.ModuleList()
        for i in range(self.num_heads):
            self.metapath_layers.append(HAN_metapath_specific(G, feature_attn_size, out_dim, dropout, slope))

        self.dropout = nn.Dropout(dropout)
        self.m_fc = nn.Linear(feature_attn_size * num_heads + m_sim_dim, out_dim)
        self.d_fc = nn.Linear(feature_attn_size * num_heads + d_sim_dim, out_dim)
        self.l_fc = nn.Linear(feature_attn_size * num_heads + l_sim_dim, out_dim)
        self.semantic_attention = SemanticAttention(in_size=out_dim * num_heads)
        self.h_fc = nn.Linear(out_dim , out_dim)
        # self.dropout = nn.Dropout(dropout)
        # self.m_fc = nn.Linear(m_sim_dim, out_dim, bias=False)
        # self.d_fc = nn.Linear(d_sim_dim, out_dim, bias=False)
        # self.fusion = nn.Linear(out_dim + feature_attn_size * num_heads, out_dim)
        self.predict = nn.Linear(out_dim * 2, 1)
        self.BilinearDecoder = BilinearDecoder(feature_size=64)
        self.InnerProductDecoder = InnerProductDecoder()

    def forward(self, G, G0, GL,GD,diseases, lncrnas):   #g_train0
        #元路径LL
        hLg = self.gatL(GL)
        #元路径DD
        hDg = self.gatD(GD)
        index1 = 0
        new_g = dgl.metapath_reachable_graph(G0, ['lm', 'ml'])
        head_outs0 = [attn_head(new_g, 'lm') for attn_head in self.metapath_layers]
        h_agg1 = torch.cat(head_outs0, dim=1)

        for meta_path in self.meta_paths:
            if meta_path == 'ld' or meta_path == 'dl':  # ll dd
                # 元路径为md和dm时，获得的聚合特征0-382是疾病特征。383-877是miRNA特征
                if index1 == 0:
                    h_agg0 = self.gat(G)
                    index1 = 1
            elif meta_path == 'lm':
                # 元路径为ml，过滤边，构建子图，利用HAN_metapath_specific获得注意力特征
                # lm_edges = G0.filter_edges(lambda edges: edges.data['lm'])
                # g_subgraph=dgl.metapath_reachable_graph(g0, ['dl', 'ld'])
                # g_lm = G0.edge_subgraph(lm_edges, preserve_nodes=True)
                new_g = dgl.metapath_reachable_graph(G0, ['lm', 'ml'])
                head_outs0 = [attn_head(new_g, meta_path) for attn_head in self.metapath_layers]
                h_agg1 = torch.cat(head_outs0, dim=1)
#             elif meta_path == 'dm':
#                 # 同元路径为dl
#                 dm_edges = G0.filter_edges(lambda edges: edges.data['dm'])
#                 g_dm = G0.edge_subgraph(dm_edges, preserve_nodes=True)
#                 head_outs1 = [attn_head(g_dm, meta_path) for attn_head in self.metapath_layers]
#                 h_agg2 = torch.cat(head_outs1, dim=1)

# 不同元路径疾病特征和不同元路径节点特征
        disease0 = h_agg0[:self.num_diseases]
        # mirna0 = h_agg0[self.num_diseases:self.num_diseases + self.num_mirnas]
        lncrna0 = h_agg0[self.num_diseases:self.num_diseases + self.num_lncrnas]
        disease1 = h_agg2[:self.num_diseases]
        # mirna1 = h_agg1[self.num_diseases:self.num_diseases + self.num_mirnas]
        lncrna1 = h_agg1[self.num_diseases:self.num_diseases + self.num_lncrnas]
        #再从这堆一个KNN的
        diseaseL =hDg[:self.num_diseases]
        lncL = hLg[self.num_diseases:self.num_diseases + self.num_lncrnas]
        semantic_embeddingsL = (lncrna0+lncL)/2
        semantic_embeddingsD = (disease0+diseaseL)/2
        # h_d:(383,895)  h_m:(495,1007)
        # semantic_embeddings1 = []
        # semantic_disease = torch.cat((disease, disease), dim=1)
# 特征在dim=1堆叠

        semantic_embeddings1 = torch.stack((semantic_embeddingsD, disease1), dim=1)
        h1 = self.semantic_attention(semantic_embeddings1)

        semantic_embeddings2 = torch.stack((semantic_embeddingsL, lncrna1), dim=1)
        h2 = self.semantic_attention(semantic_embeddings2)

# 将经过语义层注意力得到的疾病特征和miRNA特征，和原来的疾病特征和miRNA特征连接
        h_d = torch.cat((h1, self.G.ndata['d_sim'][:self.num_diseases]), dim=1)
        h_l = torch.cat((h2, self.G.ndata['l_sim'][self.num_diseases:self.num_diseases+self.num_lncrnas]), dim=1)
        # h_d = torch.cat((h1, self.G.ndata['d_sim'][:self.num_diseases]), dim=1)
        # h_l = torch.cat((h2, self.G.ndata['l_sim'][self.num_diseases:850]), dim=1)

        h_l = self.dropout(F.elu(self.l_fc(h_l)))       # （383,64）
        h_d = self.dropout(F.elu(self.d_fc(h_d)))       # （495,64）

        h = torch.cat((h_d, h_l), dim=0)    # （878,64）
        h = self.dropout(F.elu(self.h_fc(h)))

# 获取训练边或测试边的点的特征
        h_diseases = h[diseases]    # disease中有重复的疾病名称;(17376,64)
        h_lncrnas = h[lncrnas]        # (17376,64)
# 全连接层得到结果
        h_concat = torch.cat((h_diseases, h_lncrnas), 1)         # (17376,128)
        predict_score = torch.sigmoid(self.predict(h_concat))   # (17376,128)->(17376,128*2)->(17376,1)
        # return predict_score
        # predict_score = self.BilinearDecoder(h_diseases, h_lncrnas)
        # predict_score = self.InnerProductDecoder(h_diseases, h_lncrnas)
        return predict_score



In [ ]:
# %load layers.py
import torch
import torch.nn as nn
import torch.nn.functional as F

# GATLayer为了方便求解只含miRNA-疾病的聚合特征
# layers用于对比实验，判断节点层主义力相同时的情况
class GATLayer(nn.Module):
    def __init__(self, G, feature_attn_size, dropout, slope):
        super(GATLayer, self).__init__()

        self.disease_nodes = G.filter_nodes(lambda nodes: nodes.data['type'] == 1)
        self.mirna_nodes = G.filter_nodes(lambda nodes: nodes.data['type'] == 2)
        self.lncrna_nodes = G.filter_nodes(lambda nodes: nodes.data['type'] == 0)

        self.G = G
        self.slope = slope

        # self.m_fc = nn.Linear(G.ndata['m_sim'].shape[1], feature_attn_size, bias=False)
        self.d_fc = nn.Linear(G.ndata['d_sim'].shape[1], feature_attn_size, bias=False)
        self.l_fc = nn.Linear(G.ndata['l_sim'].shape[1], feature_attn_size, bias=False)

        self.dropout = nn.Dropout(dropout)
        # self.attn_fc = nn.Linear(feature_attn_size * 2, 1, bias=False)
        self.reset_parameters()

    def reset_parameters(self):
        gain = nn.init.calculate_gain('relu')   #计算给定激活函数的增益值。
        nn.init.xavier_normal_(self.l_fc.weight, gain=gain)
        nn.init.xavier_normal_(self.d_fc.weight, gain=gain)
        # nn.init.xavier_normal_(self.attn_fc.weight, gain=gain)

# 边的注意力
    def edge_attention(self, edges):
        # print('SRC size:', edges.src['z'].size())
        # print('DST size: ', edges.dst['z'].size())
        # z2 = torch.cat([edges.src['z'], edges.dst['z']], dim=1)
        # a = self.attn_fc(z2)
        # return {'e': a}
        a = torch.sum(edges.src['z'].mul(edges.dst['z']), dim=1).unsqueeze(1)
        return {'e': F.leaky_relu(a, negative_slope=self.slope)}

    def message_func(self, edges):
        return {'z': edges.src['z'], 'e': edges.data['e']}

    def reduce_func(self, nodes):
        # alpha注意力系数
        alpha = F.softmax(nodes.mailbox['e'], dim=1)
        h = torch.sum(alpha * nodes.mailbox['z'], dim=1)

        return {'h': F.elu(h)}

    def forward(self, G):   #同构图不要dropout,直接聚合

        self.G.apply_nodes(lambda nodes: {'z': self.dropout(self.d_fc(nodes.data['d_sim']))}, self.disease_nodes)
        self.G.apply_nodes(lambda nodes: {'z': self.dropout(self.l_fc(nodes.data['l_sim']))}, self.lncrna_nodes)

        self.G.apply_edges(self.edge_attention)
        self.G.update_all(self.message_func, self.reduce_func)
       
        return self.G.ndata.pop('h')

# 多头注意力
class MultiHeadGATLayer(nn.Module):  # 2
    def __init__(self, G, feature_attn_size, num_heads, dropout, slope, merge='cat'):
        super(MultiHeadGATLayer, self).__init__()

        self.G = G
        self.dropout = dropout
        self.slope = slope
        self.merge = merge

        self.heads = nn.ModuleList()
        for i in range(num_heads):  #numhead为8
            self.heads.append(GATLayer(G, feature_attn_size, dropout, slope))

    def forward(self, G):
        head_outs = [attn_head(G) for attn_head in self.heads]
        if self.merge == 'cat':
            return torch.cat(head_outs, dim=1)
        else:
            return torch.mean(torch.stack(head_outs), dim=0)

# HAN_metapath_specific用于获得元路径为ml、dl时，miRNA和疾病的特征
class HAN_metapath_specific(nn.Module):
    def __init__(self, G, feature_attn_size, out_dim, dropout, slope):
        super(HAN_metapath_specific, self).__init__()

        self.mirna_nodes = G.filter_nodes(lambda nodes: nodes.data['type'] == 2)
        self.lncrna_nodes = G.filter_nodes(lambda nodes: nodes.data['type'] == 0)
        self.disease_nodes = G.filter_nodes(lambda nodes: nodes.data['type'] == 1)
        # self.mirna_nodes = G.num_nodes('m')
        # self.lncrna_nodes = G.num_nodes('l')
        # self.disease_nodes = G.num_nodes('d')

        self.G = G
        self.slope = slope

#         self.m_fc = nn.Linear(G.ndata['m_sim'].shape[1], feature_attn_size, bias=False)
        self.l_fc = nn.Linear(G.ndata['l_sim'].shape[1], feature_attn_size, bias=False)
        self.d_fc = nn.Linear(G.ndata['d_sim'].shape[1], feature_attn_size, bias=False)


        self.m_fc1 = nn.Linear(feature_attn_size + 495, out_dim)   # 设置全连接层
        self.d_fc1 = nn.Linear(feature_attn_size + 412, out_dim)
        self.l_fc1 = nn.Linear(feature_attn_size + 240, out_dim)  ###
        self.attn_fc = nn.Linear(feature_attn_size * 2, 1, bias=False)
        self.dropout = nn.Dropout(dropout)
        self.reset_parameters()

    def reset_parameters(self):
        gain = nn.init.calculate_gain('relu')
#         nn.init.xavier_normal_(self.m_fc.weight, gain=gain)
        nn.init.xavier_normal_(self.l_fc.weight, gain=gain)
        nn.init.xavier_normal_(self.d_fc.weight, gain=gain)

    def edge_attention(self, edges):
        a = torch.sum(edges.src['z'].mul(edges.dst['z']), dim=1).unsqueeze(1)
        '''z2 = torch.cat([edges.src['z'], edges.dst['z']], dim=1)
        a = self.attn_fc(z2)'''
        return {'e': F.leaky_relu(a, negative_slope=self.slope)}

    def message_func(self, edges):
        return {'z': edges.src['z'], 'e': edges.data['e']}

    def reduce_func(self, nodes):
        alpha = F.softmax(nodes.mailbox['e'], dim=1)
        h = torch.sum(alpha * nodes.mailbox['z'], dim=1)

        return {'h': F.elu(h)}

    def forward(self, new_g, meta_path):
# 这里的图为传过来的新构建的子图
        if meta_path == 'lm':    #lm
            new_g.apply_nodes(lambda nodes: {'z': self.dropout(self.l_fc(nodes.data['l_sim']))}, self.lncrna_nodes)
            # new_g.apply_nodes(lambda nodes: {'z': self.dropout(self.l_fc(nodes.data['l_sim']))}, self.lncrna_nodes)
            # new_g.apply_nodes(lambda nodes: {'z': self.dropout(self.m_fc(nodes.data['m_sim']))}, self.mirna_nodes)
            new_g.apply_edges(self.edge_attention)
            new_g.update_all(self.message_func, self.reduce_func)

            h_lm = new_g.ndata.pop('h')

            return h_lm

        elif meta_path == 'dm':  #dm
            new_g.apply_nodes(lambda nodes: {'z': self.dropout(self.d_fc(nodes.data['d_sim']))}, self.disease_nodes)
            # new_g.apply_nodes(lambda nodes: {'z': self.dropout(self.m_fc(nodes.data['m_sim']))}, self.disease_nodes)
            new_g.apply_edges(self.edge_attention)
            new_g.update_all(self.message_func, self.reduce_func)

            h_dm = new_g.ndata.pop('h')

            return h_dm



In [ ]:
# %load Hlayers.py
import torch
import torch.nn as nn
import torch.nn.functional as F

# GATLayer为了方便求解只含miRNA-疾病的聚合特征
# layers用于对比实验，判断节点层主义力相同时的情况
class GATLayerL(nn.Module):
    def __init__(self, G, feature_attn_size, dropout, slope):
        super(GATLayerL, self).__init__()

        # self.disease_nodes = G.filter_nodes(lambda nodes: nodes.data['type'] == 1)
        # self.mirna_nodes = G.filter_nodes(lambda nodes: nodes.data['type'] == 2)
        self.lncrna_nodes = G.filter_nodes(lambda nodes: nodes.data['type'] == 0)

        self.G = G
        self.slope = slope

        # self.m_fc = nn.Linear(G.ndata['m_sim'].shape[1], feature_attn_size, bias=False)
        # self.d_fc = nn.Linear(G.ndata['d_sim'].shape[1], feature_attn_size, bias=False)
        self.l_fc = nn.Linear(G.ndata['l_sim'].shape[1], feature_attn_size, bias=False)
        self.dropout = nn.Dropout(dropout)
        # self.attn_fc = nn.Linear(feature_attn_size * 2, 1, bias=False)
        self.reset_parameters()

    def reset_parameters(self):
        gain = nn.init.calculate_gain('relu')   #计算给定激活函数的增益值。
        nn.init.xavier_normal_(self.l_fc.weight, gain=gain)
        # nn.init.xavier_normal_(self.d_fc.weight, gain=gain)
        # nn.init.xavier_normal_(self.attn_fc.weight, gain=gain)

# 边的注意力
    def edge_attention(self, edges):
        # print('SRC size:', edges.src['z'].size())
        # print('DST size: ', edges.dst['z'].size())
        # z2 = torch.cat([edges.src['z'], edges.dst['z']], dim=1)
        # a = self.attn_fc(z2)
        # return {'e': a}
        a = torch.sum(edges.src['z'].mul(edges.dst['z']), dim=1).unsqueeze(1)
        return {'e': F.leaky_relu(a, negative_slope=self.slope)}

    def message_func(self, edges):
        return {'z': edges.src['z'], 'e': edges.data['e']}

    def reduce_func(self, nodes):
        # alpha注意力系数
        alpha = F.softmax(nodes.mailbox['e'], dim=1)
        h = torch.sum(alpha * nodes.mailbox['z'], dim=1)

        return {'h': F.elu(h)}

    def forward(self, G):   #同构图不要dropout,直接聚合


        self.G.apply_nodes(lambda nodes: {'z': self.dropout(self.l_fc(nodes.data['l_sim']))}, self.lncrna_nodes)

        self.G.apply_edges(self.edge_attention)
        self.G.update_all(self.message_func, self.reduce_func)
      
        return self.G.ndata.pop('h')


class GATLayerD(nn.Module):
    def __init__(self, G, feature_attn_size, dropout, slope):
        super(GATLayerD, self).__init__()

        self.disease_nodes = G.filter_nodes(lambda nodes: nodes.data['type'] == 1)
        # self.mirna_nodes = G.filter_nodes(lambda nodes: nodes.data['type'] == 2)
        # self.lncrna_nodes = G.filter_nodes(lambda nodes: nodes.data['type'] == 0)

        self.G = G
        self.slope = slope

        # self.m_fc = nn.Linear(G.ndata['m_sim'].shape[1], feature_attn_size, bias=False)
        self.d_fc = nn.Linear(G.ndata['d_sim'].shape[1], feature_attn_size, bias=False)
        # self.l_fc = nn.Linear(G.ndata['l_sim'].shape[1], feature_attn_size, bias=False)
        self.dropout = nn.Dropout(dropout)
        # self.attn_fc = nn.Linear(feature_attn_size * 2, 1, bias=False)
        self.reset_parameters()

    def reset_parameters(self):
        gain = nn.init.calculate_gain('relu')  # 计算给定激活函数的增益值。
        # nn.init.xavier_normal_(self.l_fc.weight, gain=gain)
        nn.init.xavier_normal_(self.d_fc.weight, gain=gain)
        # nn.init.xavier_normal_(self.attn_fc.weight, gain=gain)

    # 边的注意力
    def edge_attention(self, edges):
        # print('SRC size:', edges.src['z'].size())
        # print('DST size: ', edges.dst['z'].size())
        # z2 = torch.cat([edges.src['z'], edges.dst['z']], dim=1)
        # a = self.attn_fc(z2)
        # return {'e': a}
        a = torch.sum(edges.src['z'].mul(edges.dst['z']), dim=1).unsqueeze(1)
        return {'e': F.leaky_relu(a, negative_slope=self.slope)}

    def message_func(self, edges):
        return {'z': edges.src['z'], 'e': edges.data['e']}

    def reduce_func(self, nodes):
        # alpha注意力系数
        alpha = F.softmax(nodes.mailbox['e'], dim=1)
        h = torch.sum(alpha * nodes.mailbox['z'], dim=1)

        return {'h': F.elu(h)}

    def forward(self, G):  # 同构图

        self.G.apply_nodes(lambda nodes: {'z': self.dropout(self.d_fc(nodes.data['d_sim']))}, self.disease_nodes)

        self.G.apply_edges(self.edge_attention)
        self.G.update_all(self.message_func, self.reduce_func)

        return self.G.ndata.pop('h')


# 多头注意力
class MultiHeadGATLayerL(nn.Module):  # 2
    def __init__(self, G, feature_attn_size, num_heads, dropout, slope, merge='cat'):
        super(MultiHeadGATLayerL, self).__init__()

        self.G = G
        self.dropout = dropout
        self.slope = slope
        self.merge = merge

        self.heads = nn.ModuleList()
        for i in range(num_heads):  #numhead为8
            self.heads.append(GATLayerL(G, feature_attn_size, dropout, slope))

    def forward(self, G):
        head_outs = [attn_head(G) for attn_head in self.heads]
        if self.merge == 'cat':
            return torch.cat(head_outs, dim=1)
        else:
            return torch.mean(torch.stack(head_outs), dim=0)
class MultiHeadGATLayerD(nn.Module):  # 2
    def __init__(self, G, feature_attn_size, num_heads, dropout, slope, merge='cat'):
        super(MultiHeadGATLayerD, self).__init__()

        self.G = G
        self.dropout = dropout
        self.slope = slope
        self.merge = merge

        self.heads = nn.ModuleList()
        for i in range(num_heads):  #numhead为8
            self.heads.append(GATLayerD(G, feature_attn_size, dropout, slope))

    def forward(self, G):
        head_outs = [attn_head(G) for attn_head in self.heads]
        if self.merge == 'cat':
            return torch.cat(head_outs, dim=1)
        else:
            return torch.mean(torch.stack(head_outs), dim=0)

In [ ]:
# %load model.py
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
import numpy as np
# from mxnet import ndarray as nd
# from mxnet.gluon import nn as ng


from layers import MultiHeadGATLayer, HAN_metapath_specific

# model1对比实验,语义层贡献不考虑，将beta设为一个定值
class SemanticAttention(nn.Module):
    def __init__(self, in_size, hidden_size=512):
        super(SemanticAttention, self).__init__()

        self.project = nn.Sequential(
            nn.Linear(in_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, 1, bias=False)
        )

    def forward(self, z):
        w = self.project(z).mean(0)
        # beta = torch.softmax(w, dim=0)
        beta = torch.sigmoid(w)
        beta = beta.expand((z.shape[0],) + beta.shape)
        # delta = beta * z
        return (0.5 * z).sum(1)     # 这里设置为0.5为了保证元路径贡献相同

class HGANMDA(nn.Module):
    def __init__(self, G, meta_paths_list, feature_attn_size, num_heads, num_diseases, num_mirnas, num_lncrnas,
                 d_sim_dim, m_sim_dim, l_sim_dim, out_dim, dropout, slope):
        super(HGANMDA, self).__init__()

        self.G = G
        self.meta_paths = meta_paths_list
        self.num_heads = num_heads
        self.num_diseases = num_diseases
        self.num_mirnas = num_mirnas
        self.num_lncrnas = num_lncrnas

        self.gat = MultiHeadGATLayer(G, feature_attn_size, num_heads, dropout, slope)  #计算节点间的多头注意力
        self.heads = nn.ModuleList()

        self.metapath_layers = nn.ModuleList()
        for i in range(self.num_heads):
            self.metapath_layers.append(HAN_metapath_specific(G, feature_attn_size, out_dim, dropout, slope))

        self.dropout = nn.Dropout(dropout)
        self.m_fc = nn.Linear(feature_attn_size * num_heads + m_sim_dim, out_dim)
        self.d_fc = nn.Linear(feature_attn_size * num_heads + d_sim_dim, out_dim)
        self.semantic_attention = SemanticAttention(in_size=out_dim * num_heads)   #语义层
        self.h_fc = nn.Linear(out_dim , out_dim)
        # self.dropout = nn.Dropout(dropout)
        # self.m_fc = nn.Linear(m_sim_dim, out_dim, bias=False)
        # self.d_fc = nn.Linear(d_sim_dim, out_dim, bias=False)
        # self.fusion = nn.Linear(out_dim + feature_attn_size * num_heads, out_dim)
        self.predict = nn.Linear(out_dim * 2, 1)
        self.BilinearDecoder = BilinearDecoder(feature_size=64)
        self.InnerProductDecoder = InnerProductDecoder()

    def forward(self, G, G0, diseases, mirnas):

        index1 = 0
        for meta_path in self.meta_paths:
            if meta_path == 'md' or meta_path == 'dm':
                if index1 == 0:
                    h_agg0 = self.gat(G)
                    index1 = 1
            elif meta_path == 'ml':
                ml_edges = G0.filter_edges(lambda edges: edges.data['ml'])
                g_ml = G0.edge_subgraph(ml_edges, preserve_nodes=True)
                head_outs0 = [attn_head(g_ml, meta_path) for attn_head in self.metapath_layers]
                h_agg1 = torch.cat(head_outs0, dim=1)
            elif meta_path == 'dl':
                dl_edges = G0.filter_edges(lambda edges: edges.data['dl'])
                g_dl = G0.edge_subgraph(dl_edges, preserve_nodes=True)
                head_outs1 = [attn_head(g_dl, meta_path) for attn_head in self.metapath_layers]
                h_agg2 = torch.cat(head_outs1, dim=1)

        disease0 = h_agg0[:383]
        mirna0 = h_agg0[383:878]
        disease1 = h_agg2[:383]
        mirna1 = h_agg1[383:878]
        # h_d:(383,895)  h_m:(495,1007)
        # semantic_embeddings1 = []
        # semantic_disease = torch.cat((disease, disease), dim=1)
        semantic_embeddings1 = torch.stack((disease0, disease1), dim=1)
        h1 = self.semantic_attention(semantic_embeddings1)
        semantic_embeddings2 = torch.stack((mirna0, mirna1), dim=1)
        h2 = self.semantic_attention(semantic_embeddings2)

        h_d = torch.cat((h1, self.G.ndata['d_sim'][:self.num_diseases]), dim=1)
        h_m = torch.cat((h2, self.G.ndata['m_sim'][self.num_diseases:878]), dim=1)

        h_m = self.dropout(F.elu(self.m_fc(h_m)))       # （383,64）
        h_d = self.dropout(F.elu(self.d_fc(h_d)))       # （495,64）

        h = torch.cat((h_d, h_m), dim=0)    # （878,64）
        h = self.dropout(F.elu(self.h_fc(h)))

        h_diseases = h[diseases]    # disease中有重复的疾病名称;(17376,64)
        h_mirnas = h[mirnas]        # (17376,64)

        # h_concat = torch.cat((h_diseases, h_mirnas), 1)         # (17376,128)
        # predict_score = torch.sigmoid(self.predict(h_concat))   # (17376,128)->(17376,128*2)->(17376,1)
        # return predict_score
        #predict_score = self.BilinearDecoder(h_diseases, h_mirnas)
        predict_score = self.InnerProductDecoder(h_diseases, h_mirnas)
        return predict_score

# 双线性解码器
class BilinearDecoder(nn.Module):
    def __init__(self, feature_size):
        super(BilinearDecoder, self).__init__()

        # self.activation = ng.Activation('sigmoid')  # 定义sigmoid激活函数
        # 获取维度为(embedding_size, embedding_size)的参数矩阵，即论文中的Q参数矩阵

        self.W = Parameter(torch.randn(feature_size, feature_size))

    def forward(self, h_diseases, h_mirnas):
        h_diseases0 = torch.mm(h_diseases, self.W)
        h_mirnas0 = torch.mul(h_diseases0, h_mirnas)
        # h_mirnas0 = h_mirnas.tanspose(0,1)
        # h_mirnsa0 = torch.mm(h_diseases0, h_mirnas0)
        h0 = h_mirnas0.sum(1)
        h = torch.sigmoid(h0)
        h = h.unsequence(1)
        return h

# 内积解码器
class InnerProductDecoder(nn.Module):
    """Decoder model layer for link prediction."""
    def __init__(self):
        super(InnerProductDecoder, self).__init__()

    def forward(self, h_diseases, h_mirnas):
        x = torch.mul(h_diseases, h_mirnas).sum(1)
        x = torch.reshape(x, [-1])
        outputs = F.sigmoid(x)
        return outputs



In [ ]:
# %load train.py
import time
import numpy as np
import pandas as pd
import random
import torch
import torch.nn as nn
from sklearn.model_selection import KFold
from sklearn import metrics

from utils import load_data, build_graph, weight_reset
from model1 import HGANMDA


def Train(directory, epochs, attn_size, attn_heads, out_dim, dropout, slope, lr, wd, random_seed, cuda, model_type):
    random.seed(random_seed)
    np.random.seed(random_seed)
    torch.manual_seed(random_seed)
    if torch.cuda.is_available():
        if not cuda:
            print('WARNING: You have a CUDA device, so you should probably run with --cuda')
        else:
            torch.cuda.manual_seed(random_seed)

    # if cuda:
    #     context = torch.device('cuda:0')
    # else:
    #     context = torch.device('cpu')
    context = torch.device('cpu')
    # g为miRNA和disease图，g0为miRNA、disease和lncRNA图
    g, g0, gL, gD,disease_vertices, lncrna_vertices, ID, IM, IL, samples, ml_associations, ld_associations  ,ll_associations ,dd_associations= build_graph(directory, random_seed)
    samples_df = pd.DataFrame(samples, columns=['disease', 'lncRNA', 'label'])
    case=np.zeros(240)
    for i in range(240):
        case[i]=2
    # case_train_tensor=torch.from_numpy(case.astype('int64'))
    case_disease=[]
    case_lncRNA=[]
    for i in range(240):
        case_disease.append(251)
        case_lncRNA.append(i+412)
    print('## vertices:', g.number_of_nodes())
    print('## edges:', g.number_of_edges())
    print('## disease nodes:', torch.sum(g.ndata['type'] == 1).numpy())
    print('## lncrna nodes g: ', torch.sum(g.ndata['type'] == 0).numpy())
    print('## mirna nodes: ', g0.num_nodes('m'))
    print('## lncrna nodes: ', g0.num_nodes('l'))
    # print('## mirna nodes: ', torch.sum(g0.ndata['type'] == 2).numpy())
    # print('## lncrna nodes: ', torch.sum(g0.ndata['type'] == 0).numpy())
    g.to(context)

    auc_result = []
    acc_result = []
    pre_result = []
    recall_result = []
    f1_result = []
    prc_result = []

    fprs = []
    tprs = []
    precisions = []
    recalls = []

# 设置五折交叉验证
    i = 0
    kf = KFold(n_splits=5, shuffle=True, random_state=random_seed)

    for train_idx, test_idx in kf.split(samples[:, 2]):
        i += 1
        print('Training for Fold', i)
# 将训练集的指针标记为1，其余为0
        samples_df['train'] = 0
        samples_df['train'].iloc[train_idx] = 1       #(d,l)


        train_tensor = torch.from_numpy(samples_df['train'].values.astype('int64'))
        edge_data = {'train': train_tensor}
# 对两个异质图的训练边进行标记
        g.edges[disease_vertices, lncrna_vertices].data.update(edge_data)
        g.edges[lncrna_vertices, disease_vertices].data.update(edge_data)

        # case_edge_data = {'train': case_train_tensor}
        g.add_edges(case_disease, case_lncRNA,data={'case': torch.from_numpy(case.astype('int64'))})
        case_test_eid = g.filter_edges(lambda edges: edges.data['case']==2 )
        case_src_test, case_dst_test = g.find_edges(case_test_eid)

        lnc_V=lncrna_vertices.copy()
        for i,v in enumerate(lncrna_vertices):
            lnc_V[i]=v-ID.shape[0]

        gL.edges[lnc_V, lnc_V].data.update(edge_data)
        gD.edges[disease_vertices, disease_vertices].data.update(edge_data)
        # g0.edges[disease_vertices, lncrna_vertices].data.update(edge_data)
        # g0.edges[lncrna_vertices, disease_vertices].data.update(edge_data)
        # 由于图的边的添加训练的原因，g0和g不包含lncRNA节点的边的标记相同，然后分别根据训练边构建子图
        # 由于图的边的添加训练的原因，g0和g不包含miRNA节点的边的标记相同，然后分别根据训练边构建子图
        train_eid = g.filter_edges(lambda edges: edges.data['train'])
        # trainid_l= gL.filter_edges(lambda edges: edges.data['train'])
        # trainid_d = gD.filter_edges(lambda edges: edges.data['train'])
        g_train = g.edge_subgraph(train_eid, preserve_nodes=True)
        # gL_train0 = gL.edge_subgraph(trainid_l, preserve_nodes=True)
        # gD_train0 = gD.edge_subgraph(trainid_d, preserve_nodes=True)
        # g_train0 = g0.edge_subgraph(train_eid, preserve_nodes=True)
        # g_train.copy_from_parent() dgl0.4之后的版本没有这个语句

        # 训练标签和测试标签用于最后对比
        label_train = g_train.edata['label'].unsqueeze(1)
        src_train, dst_train = g_train.all_edges()

        test_eid = g.filter_edges(lambda edges: edges.data['train'] == 0)
        src_test, dst_test = g.find_edges(test_eid)                    ##

        label_test = g.edges[test_eid].data['label'].unsqueeze(1)


        print('## Training edges:', len(train_eid))
        print('## Testing edges:', len(test_eid))

        if model_type == 'MMHGAN':
            model = HGANMDA(G=g_train,              #g_train0
                            GL=gL,
                            GD=gD,
                           meta_paths_list=['ld', 'lm', 'dm', 'dl'],    # 'md', 'dm', 'ml', 'dl'
                           feature_attn_size=attn_size,
                           num_heads=attn_heads,
                           num_diseases=ID.shape[0],
                           num_mirnas=IM.shape[0],
                           num_lncrnas=IL.shape[0],
                           d_sim_dim=ID.shape[1],
                           m_sim_dim=IM.shape[1],
                           l_sim_dim=IL.shape[1],
                           out_dim=out_dim,
                           dropout=dropout,
                           slope=slope,
                           )


        model.apply(weight_reset)
        model.to(context)
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
        loss = nn.BCELoss()

        for epoch in range(epochs):
            start = time.time()

            model.train()
            with torch.autograd.set_detect_anomaly(True):
                # score_train = model(g_train0, g0, src_train, dst_train)   #
                score_train = model(g_train, g0, gL , gD ,src_train, dst_train)
                loss_train = loss(score_train, label_train)

                optimizer.zero_grad()
                loss_train.backward()
                optimizer.step()

            model.eval()
            with torch.no_grad():
                # score_val = model(g, g0, src_test, dst_test)    #
                score_val = model(g, g0,gL ,gD , src_test, dst_test)
                loss_val = loss(score_val, label_test)

            score_train_cpu = np.squeeze(score_train.cpu().detach().numpy())
            score_val_cpu = np.squeeze(score_val.cpu().detach().numpy())
            label_train_cpu = np.squeeze(label_train.cpu().detach().numpy())
            label_val_cpu = np.squeeze(label_test.cpu().detach().numpy())

            train_auc = metrics.roc_auc_score(label_train_cpu, score_train_cpu)
            val_auc = metrics.roc_auc_score(label_val_cpu, score_val_cpu)

            pred_val = [0 if j < 0.5 else 1 for j in score_val_cpu]
            acc_val = metrics.accuracy_score(label_val_cpu, pred_val)
            pre_val = metrics.precision_score(label_val_cpu, pred_val)
            recall_val = metrics.recall_score(label_val_cpu, pred_val)
            f1_val = metrics.f1_score(label_val_cpu, pred_val)

            end = time.time()

            if (epoch + 1) % 10 == 0:
                print('Epoch:', epoch + 1, 'Train Loss: %.4f' % loss_train.item(),
                      'Val Loss: %.4f' % loss_val.cpu().detach().numpy(),
                      'Acc: %.4f' % acc_val, 'Pre: %.4f' % pre_val, 'Recall: %.4f' % recall_val, 'F1: %.4f' % f1_val,
                      'Train AUC: %.4f' % train_auc, 'Val AUC: %.4f' % val_auc, 'Time: %.2f' % (end - start))

        model.eval()
        with torch.no_grad():
            # score_test = model(g, g0, src_test, dst_test)  #
            score_test = model(g, g0, gL,gD,src_test, dst_test)  #
            case_test=model(g, g0, gL,gD,case_src_test, case_dst_test)

        score_test_cpu = np.squeeze(score_test.cpu().detach().numpy())
        label_test_cpu = np.squeeze(label_test.cpu().detach().numpy())    #打印结果
        case_test_cpu = np.squeeze(case_test.cpu().detach().numpy())
        print(case_test_cpu)
        # case_test_cpu= pandas.DataFrame(data1)
        # # data1.to_csv('D.csv',index=None,header=False)


        fpr, tpr, thresholds = metrics.roc_curve(label_test_cpu, score_test_cpu)
        precision, recall, _ = metrics.precision_recall_curve(label_test_cpu, score_test_cpu)
        test_auc = metrics.auc(fpr, tpr)
        test_prc = metrics.auc(recall, precision)

        pred_test = [0 if j < 0.5 else 1 for j in score_test_cpu]
        acc_test = metrics.accuracy_score(label_test_cpu, pred_test)
        pre_test = metrics.precision_score(label_test_cpu, pred_test)
        recall_test = metrics.recall_score(label_test_cpu, pred_test)
        f1_test = metrics.f1_score(label_test_cpu, pred_test)

        print('Fold: ', i, 'Test acc: %.4f' % acc_test, 'Test Pre: %.4f' % pre_test,
              'Test Recall: %.4f' % recall_test, 'Test F1: %.4f' % f1_test, 'Test PRC: %.4f' % test_prc,
              'Test AUC: %.4f' % test_auc)

        auc_result.append(test_auc)
        acc_result.append(acc_test)
        pre_result.append(pre_test)
        recall_result.append(recall_test)
        f1_result.append(f1_test)
        prc_result.append(test_prc)

        fprs.append(fpr)
        tprs.append(tpr)
        precisions.append(precision)
        recalls.append(recall)

    print('## Training Finished !')
    print('-----------------------------------------------------------------------------------------------')
    print('-AUC mean: %.4f, variance: %.4f \n' % (np.mean(auc_result), np.std(auc_result)),
          'Accuracy mean: %.4f, variance: %.4f \n' % (np.mean(acc_result), np.std(acc_result)),
          'Precision mean: %.4f, variance: %.4f \n' % (np.mean(pre_result), np.std(pre_result)),
          'Recall mean: %.4f, variance: %.4f \n' % (np.mean(recall_result), np.std(recall_result)),
          'F1-score mean: %.4f, variance: %.4f \n' % (np.mean(f1_result), np.std(f1_result)),
          'PRC mean: %.4f, variance: %.4f \n' % (np.mean(prc_result), np.std(prc_result)))

    return fprs, tprs, auc_result, precisions, recalls, prc_result

In [ ]:
# %load main.py
import warnings

from train import Train
from utils import plot_auc_curves, plot_prc_curves


if __name__ == '__main__':
    warnings.filterwarnings("ignore")

    fprs, tprs, auc, precisions, recalls, prc = Train(directory='new',
                                                      epochs=10,           #100 beta
                                                      attn_size=128,
                                                      attn_heads=4,
                                                      out_dim=128,
                                                      dropout=0.5,
                                                      slope=0.2,
                                                      lr=0.001,
                                                      wd=5e-3,
                                                      random_seed=1234,
                                                      cuda=True,
                                                      model_type='MMHGAN)  # GATMDA_only_attn   GATMDA_without_attn

    plot_auc_curves(fprs, tprs, auc, directory='roc_result', name='test_auc')
    plot_prc_curves(precisions, recalls, prc, directory='roc_result', name='test_prc')